In [26]:
import pandas as pd
import opendatasets as od


In [27]:
# od.download("https://www.kaggle.com/competitions/spaceship-titanic")

In [28]:
raw_data=pd.read_csv('spaceship-titanic/train.csv')
raw_data=raw_data[~raw_data['Cabin'].isnull()]





In [29]:
extract_cabin=raw_data.Cabin.str.split('/',expand=True)
raw_data=raw_data.assign(Deck=extract_cabin[0])

In [30]:
ytrain=raw_data['Transported']

In [31]:


def clean_dt(raw_data):
    return (raw_data
    .drop(columns=['Name','PassengerId','Destination','HomePlanet','Age'])
    .assign(
        RoomService=raw_data.RoomService.fillna(0).replace('nan',0),
        CryoSleep=raw_data.CryoSleep.fillna(False),
        VIP=raw_data.VIP.fillna(False).replace('nan',0),
        FoodCourt=raw_data.FoodCourt.fillna(0).replace('nan',0),
        ShoppingMall=raw_data.ShoppingMall.fillna(0).replace('nan',0),
        Spa=raw_data.Spa.fillna(0).replace('nan',0),
        VRDeck=raw_data.VRDeck.fillna(0).replace('nan',0),
        Cabin=raw_data.Cabin.str.split('/',expand=True)[2]=='S',
        Age=raw_data.Age.fillna(raw_data.Age.median())
        )
     
    )



In [32]:
 # encoding boolean labels
from sklearn import preprocessing

def transform_step(raw_data):
    label_encoder=preprocessing.LabelEncoder()
    raw_data.CryoSleep=label_encoder.fit_transform(raw_data.CryoSleep).astype(int)
    raw_data.Cabin=label_encoder.fit_transform(raw_data.Cabin).astype(int)
    # raw_data.VIP=label_encoder.fit_transform(raw_data.VIP)
    # raw_data=pd.get_dummies(raw_data,columns=['Destination','HomePlanet'])
    raw_data=pd.get_dummies(raw_data,columns=['Deck'])

    return raw_data

In [33]:
res_encoder=preprocessing.LabelEncoder()

ytrain=res_encoder.fit_transform(ytrain)

In [34]:
  
Xtrain=clean_dt(raw_data)
Xtrain=transform_step(Xtrain)
Xtrain.drop(columns=['Transported'],inplace=True)   

# cabins=Xtrain.Cabin.str.split('/',expand=True),
# cabins

In [35]:
raw_test=pd.read_csv('spaceship-titanic/test.csv')

extract_cabin=raw_test.Cabin.str.split('/',expand=True)
raw_test=raw_test.assign(Deck=extract_cabin[0])

Xtest=clean_dt(raw_test)
Xtest=transform_step(Xtest)

In [36]:
# Xtrain.Cabin1=res_encoder.fit_transform(Xtrain.Cabin1).astype(int)
# Xtest.Cabin1=res_encoder.fit_transform(Xtest.Cabin1).astype(int)

In [37]:
Xtrain.dtypes

CryoSleep         int32
Cabin             int32
VIP                bool
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Age             float64
Deck_A            uint8
Deck_B            uint8
Deck_C            uint8
Deck_D            uint8
Deck_E            uint8
Deck_F            uint8
Deck_G            uint8
Deck_T            uint8
dtype: object

In [38]:
Xtrain.drop(columns=['Deck_A','Deck_D','Deck_E','Deck_F','Deck_G','Deck_T'],inplace=True)
Xtest.drop(columns=['Deck_A','Deck_D','Deck_E','Deck_F','Deck_G','Deck_T'],inplace=True)

In [39]:
(Xtrain
 .assign(Transported=ytrain==True)
 .corr(method='spearman')
 .style
 .background_gradient(cmap='RdBu',vmax=1,vmin=-1)
 .set_sticky(axis='index')
)

,CryoSleep,Cabin,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Age,Deck_B,Deck_C,Transported
CryoSleep,1.000000,0.020889,-0.076476,-0.507133,-0.521453,-0.504342,-0.537715,-0.515993,-0.072007,0.126918,0.029940,0.459200
Cabin,0.020889,1.000000,-0.010163,-0.021415,0.008081,-0.015791,0.000925,-0.010559,0.010176,0.026711,0.024023,0.103775
VIP,-0.076476,-0.010163,1.000000,0.048794,0.108646,0.034727,0.086425,0.094959,0.102628,0.066517,0.067025,-0.035086
RoomService,-0.507133,-0.021415,0.048794,1.000000,0.180158,0.428001,0.239585,0.179149,0.123040,-0.147660,-0.096716,-0.362576
FoodCourt,-0.521453,0.008081,0.108646,0.180158,1.000000,0.186424,0.468248,0.494490,0.199336,0.064883,0.177280,-0.184313
ShoppingMall,-0.504342,-0.015791,0.034727,0.428001,0.186424,1.000000,0.255240,0.187024,0.099145,-0.127907,-0.093481,-0.227038
Spa,-0.537715,0.000925,0.086425,0.239585,0.468248,0.255240,1.000000,0.435542,0.191256,0.013277,0.117535,-0.365141
VRDeck,-0.515993,-0.010559,0.094959,0.179149,0.494490,0.187024,0.435542,1.000000,0.176268,0.036905,0.144203,-0.347117
Age,-0.072007,0.010176,0.102628,0.123040,0.199336,0.099145,0.191256,0.176268,1.000000,0.109107,0.143197,-0.071239
Deck_B,0.126918,0.026711,0.066517,-0.147660,0.064883,-0.127907,0.013277,0.036905,0.109107,1.000000,-0.098672,0.146571


It seems that the persons who have billed luxury amenities have a negative relationship with being transported. In simple words, the people who have bills are less likely to be transported.

In [40]:
# prediction
import xgboost as xgb

In [140]:
# test train split
from sklearn import model_selection

kag_X_train,kag_X_test,kag_y_train,kag_y_test=model_selection.train_test_split(Xtrain,ytrain,test_size=.3,random_state=2,stratify=ytrain)

xg=xgb.XGBClassifier(early_stopping_rounds=20,eval_metric='error')
xg.fit(kag_X_train,kag_y_train,eval_set=[(kag_X_train,kag_y_train),(kag_X_test,kag_y_test)])
xg.score(kag_X_test,kag_y_test)
# res=xg.predict(Xtest)

[0]	validation_0-error:0.21043	validation_1-error:0.22401
[1]	validation_0-error:0.19966	validation_1-error:0.21420
[2]	validation_0-error:0.19546	validation_1-error:0.20518
[3]	validation_0-error:0.19327	validation_1-error:0.20361
[4]	validation_0-error:0.19209	validation_1-error:0.20439
[5]	validation_0-error:0.19226	validation_1-error:0.19890
[6]	validation_0-error:0.18923	validation_1-error:0.19576
[7]	validation_0-error:0.18671	validation_1-error:0.19694
[8]	validation_0-error:0.18604	validation_1-error:0.19694
[9]	validation_0-error:0.18755	validation_1-error:0.19576
[10]	validation_0-error:0.18705	validation_1-error:0.19459
[11]	validation_0-error:0.18167	validation_1-error:0.19733
[12]	validation_0-error:0.18167	validation_1-error:0.19733
[13]	validation_0-error:0.18049	validation_1-error:0.19733
[14]	validation_0-error:0.17813	validation_1-error:0.19772
[15]	validation_0-error:0.17813	validation_1-error:0.19772
[16]	validation_0-error:0.17746	validation_1-error:0.19812
[17]	va

0.8054138877991369

In [136]:
res=xg.predict(Xtest)

In [139]:
xg.best_ntree_limit

11

In [43]:
# random_for=xgb.XGBClassifier()
model1=xgb.XGBClassifier(n_estimators=7,max_depth=8)


model1.fit(Xtrain,ytrain)
res=model1.predict(Xtest)

In [137]:
# submission
result=pd.DataFrame({'PassengerId':raw_test.PassengerId,'Transported':res_encoder.inverse_transform(res)})
result.to_csv('submission2.csv',index=False)